In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from PIL import Image

device = "cuda:0"

modelPath = "nikravan/glm-4vq"
tokenizer = AutoTokenizer.from_pretrained(modelPath, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    modelPath,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto",
    use_cache=True,
)

/home/boom/.pyenv/versions/aivet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Loading checkpoint shards: 100%|██████████| 5/5 [00:02<00:00,  1.84it/s]


ValueError: `.to` is not supported for `4-bit` or `8-bit` bitsandbytes models. Please use the model as it is, since the model has already been set to the correct devices and casted to the correct `dtype`.

In [ ]:
from transformers import pipeline

nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-document-qa"
)

In [ ]:
image = Image.open("../data/input/Analisi Hope pagina 1.jpg").convert("RGB")

In [ ]:
query = """Task: Extract and list all blood test details.

Objective:
1. Identify and extract **chemical names** from the blood test report.
2. Capture the **detected levels** of each chemical.
3. Extract the **normal range** for each corresponding chemical.

Expected Output Format:
- Chemical Name: [Name]
  - Detected Level: [Value]
  - Normal Range: [Min Value] - [Max Value]

Please ensure accurate extraction, including any unit symbols (e.g., mg/dL, IU/L), and handle variations in formatting or alignment. Return results in a structured list format for easy readability."""

In [ ]:
nlp(image, query, temperature = 0.2, max_length = 2500, top_k =  1)

In [ ]:
query = """Task: Extract and list all blood test details.

Objective:
1. Identify and extract **chemical names** from the blood test report.
2. Capture the **detected levels** of each chemical.
3. Extract the **normal range** for each corresponding chemical.

Expected Output Format:
- Chemical Name: [Name]
  - Detected Level: [Value]
  - Normal Range: [Min Value] - [Max Value]

Please ensure accurate extraction, including any unit symbols (e.g., mg/dL, IU/L), and handle variations in formatting or alignment. Return results in a structured list format for easy readability."""
# image=""
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

In [ ]:
query = "Extract all the blood test chemical name values, their detected level and the corresponding normal range"
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

In [ ]:
query = "Extract all the blood test chemical name values, their detected level and the corresponding normal range only for test values which have abnormal levels."
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))

In [ ]:
query = "Based on the detected test values in the blood test document as well as the reported reference ranges, identify and list the molecules which are outside the reference ranges. List the molecule name, value and the reference range. Finally perform a health diagnosis based on the abnormalities. Do not list any molecules which are within the reference range, those are considered normal."
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "image": image, "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True,
)  # chat with image mode

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature": 0.2}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs["input_ids"].shape[1] :]
    print(tokenizer.decode(outputs[0]))